Reflector Area Processing
=========================



To test the effect of different degrees of gap filling, this notebook was run with `dilate_erode` set to each of [0,10,20] (where 0, None and False are functionally the same). The results of these are displayed statically in the notebook in tables - however, this requires the notebook to be run with `dilate_erode` set to each of 0, 10 and 20 beforehand.



In [1]:
dilate_erode = False
# Whether to overwrite pre-existing data or not.
FORCE_OVERWRITE = False

####################################
# Don't change anything below here #
####################################
if dilate_erode:
    file_prepend = f"modified-{dilate_erode}"
else:
    file_prepend = "unmodified"

# General util funcs as detailed in ../section_scans-example/working.org (or its derivatives)
from util_funcs import *
from plotting import *

import os
def save_figure(path):
    ''' Check if an image containing figure output already exists, otherwise save that figure.

    path (string) : path to save the figure to.

    returns None
    '''
    if not os.path.exists(path) or FORCE_OVERWRITE:
        plt.savefig(path,bbox_inches="tight")
    return

import pandas as pd
import json

# Degree of alteration assigned to each section.
alteration_degree = {"M04":0,
                     "07A":0,
                     "M08":0,
                     "06C":1,
                     "M07B1":1,
                     "M07B2":1,
                     "M01":1,
                     "M02":2}

# Text descriptions for each level of alteration.
alteration_desc = {0:"Partly",
                   1:"Heavily",
                   2:"V. Heavily"}

PLT = Plotter(alteration_degree,alteration_desc)
plot_all = PLT.plot_all

## Area Processing



Using methods detailed in `../section_scans-example/working.org` (or its derivatives), captured by the class `AreaProcessor` in `area_processing.py`

Extracting area contours into .npy files and extracting areas into `areas.json`.

-   `areas.json` is a very large file without linebreaks and so shouldn't be opened with text editors.

As an update/difference to the example processing notebook, the largest grain is also added to the data in `areas.json`.



In [1]:
from area_processing import *

pix2mm = 1/1000

# Folder containing thresholded samples (stored as .png).
samples_dir = os.path.join("..","..","DATASETS","RL_scans")

# Load thresholded sample filenames from the folder.
samples = [f for f in os.listdir(samples_dir) if f.endswith(".png")]

# Check if the areas datafile needs to be regenerated on the basis of missing file or request.
# The areas datafile is specific to the processing pathway used to compute the areas (in terms of how much dilation-erosion is applied).
if not os.path.exists(file_prepend + "-areas.json") or FORCE_OVERWRITE:
    # Declare dictionary in which areas data will be stored.
    areas_data = dict()
    print("(Re)Generating areas.json ...")
    # Check if the folder for storing filtered data in is present, and if not, create this folder.
    filtered_data_dir = "filtered_data"
    if not os.path.exists(filtered_data_dir):
        os.mkdir(filtered_data_dir)
    # Iterate through the samples with thresholded reflectors as identified above.
    for sample in samples:
        print(f"Looking at {sample}")
        # Initiate area processor for the active sample, conversion pixels to mm conversion factor and desired processing pathway.
        AP = AreaProcessor(os.path.join(samples_dir,sample),pix2mm,dilate_erode)
        # Retrieve contours.
        contours,larger_contours = AP.load_contours()
        # Retrieve patch areas.
        patch_areas,units = AP.find_areas()
        # Find the largest grain area before filtering.
        largest_grain = max(patch_areas)
        # Size filtering (selecting only areas larger than 5 pixels, and smaller than 0.05 mm2).
        min_reflector_area = 5 * pix2mm**2 # mm2
        max_reflector_area = 0.05 # mm2
        # Construct boolean filter based on grain size.
        size_filter = construct_minmax_filter(patch_areas,min_reflector_area,max_reflector_area)
        # Filter the patch areas using this boolean filter.
        patch_areas = patch_areas[size_filter]
        # Filter "small" and "large" contours using this boolean filter.
        contours = list_of_list_filter(contours,size_filter)
        larger_contours = list_of_list_filter(larger_contours,size_filter)
        # Save the filtered contours if their savefiles aren't already present.
        base_data_file = os.path.join(filtered_data_dir,f"{file_prepend}-{sample}")
        if not os.path.exists(base_data_file + ".npy"):
            np.save(base_data_file + ".npy",np.array(contours,dtype=object))
            np.save(base_data_file + "-larger.npy",np.array(larger_contours,dtype=object))

        # Extract sample name from sample filename.
        sample = sample.replace(".png","")
        # Construct dictionary to place sample-specific area data.
        areas_data[sample] = dict()
        # Add reflector patch areas.
        areas_data[sample]["patch_areas"] = list(patch_areas)
        # Add the area considered when looking at patch areas.
        areas_data[sample]["area_studied"] = AP.area_studied()
        # Add the largest grain observed.
        areas_data[sample]["largest_grain"] = largest_grain
    # Save all samples' areas data for this processing pathway.
    with open(file_prepend + "-areas.json","w") as outfile:
        json.dump(areas_data,outfile)
else:
    print(f"Loading {file_prepend}-areas.json")
    # Load data from persistent storage.
    with open(file_prepend + "-areas.json") as infile:
        areas_data = json.load(infile)
print("... complete")

None

### Area Distribution Plotting



On the plots, the area range (x-axis) is hardcoded (to between 0 and 0.05 mm<sup>2</sup>).



In [1]:
fig = plot_all(PLT.area_distros,file_prepend)
fig.suptitle("Area Distributions")
save_figure(os.path.join("imgs",file_prepend + "-area-distro.png"))
plt.show()

None

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-area-distro.png"></th>
<th><img src="./imgs/modified-10-area-distro.png"></th>
<th><img src="./imgs/modified-20-area-distro.png"></th>
</tr>
</table>



#### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:04&gt;</span></span>
Observations:

-   The main difference between partially and heavily altered is that the heavily altered distributions appear to overall have broader distributions.
-   Increasing dilation-erosionn appears to broaden the distributions.
    -   This effect is particularly pronounced for M02, a very (?) heavily altered sample.

Interpretations

-   Increased alteration increases growth of reflectors, biasing them towards larger sizes.
-   Dilation-erosion causes joining of grains that don't get separated by erosion, and hence a general increase in size. This effect should be amplified if there are lots of reflector grain clusters. Therefore M02 likely has a lot of reflector grain clusters.



## Reflector Area vs Nearest Neighbour Distance



On the plots, the area range (x-axis) is hardcoded (to between 0 and 0.05 mm<sup>2</sup>), and the nearest neighbour distance is hardcoded (to between 0 and 1 mm).



In [1]:
fig = plot_all(PLT.area_vs_nn_dist,file_prepend)
fig.suptitle("Area vs Nearest Neighbour Distance")
save_figure(os.path.join("imgs",file_prepend + "-area-nn-dist.png"))
plt.show()

None

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-area-nn-dist.png"></th>
<th><img src="./imgs/modified-10-area-nn-dist.png"></th>
<th><img src="./imgs/modified-20-area-nn-dist.png"></th>
</tr>
</table>



### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:06&gt;</span></span>
Observations:

-   There's a large spread of nearest-neighbour distances for the finest grains; as grains become larger, nearest-neighbour distance appears to converge to a value around 0.1-0.2 mm.
-   Increasing dilation-erosion increases the modal separation distance (the peak in the distributions of nearest-neighbour distance).
-   There are more larger grains with increasing dilation-erosion, which means the convergence is clearer.

Interpretations:

-   Increasing dilation-erosion means grains will generally grow in size, such that a lot of low-separation fine grains become merged, hence the increase in modal separation and spreading out towards larger grain sizes.



## Reflector Aspect Ratios



On the plots, the aspect ratio range (x-axis) is hardcoded (to between 0 and 20).



In [1]:
fig = plot_all(PLT.aspect_ratio_distros,file_prepend)
fig.suptitle("Aspect Ratio Distributions")
save_figure(os.path.join("imgs",file_prepend + "-aspect-ratios.png"))
plt.show()

None

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-aspect-ratios.png"></th>
<th><img src="./imgs/modified-10-aspect-ratios.png"></th>
<th><img src="./imgs/modified-20-aspect-ratios.png"></th>
</tr>
</table>



### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:10&gt;</span></span>
Observations:

-   The modal aspect ratio is nearest to 1.
-   Aspect ratios are quite variable within each collection of samples with common degrees of alteration.
-   In some cases increasing dilation-erosion broadens the distribution of aspect ratios (07A from 10x10 px to 20x20 px), in others it narrows (07A from no dilation-erosion to 10x10 px), and in others it has little effect (M04, M07B1 with increasing dilation erosion), with this effect being inconsistent.

Interpretation:

-   There's probably no confident information that can be extracted from these distributions due to a lack of consistency.



## Generalised Section Properties Processing



The generalised section properties (table [1](#orgc1170a9)) are section-specific (as opposed to grain-specific) properties that were initially though to be useful to compare between sections.


| Property|Description|Units|
|---|---|---|
| <code>convhull</code>|area studied|mm<sup>2</sup>|
| <code>n</code>|number of reflectors considered||
| <code>total_area</code>|total area covered by reflectors|mm<sup>2</sup>|
| <code>largest</code>|area of largest reflector|mm<sup>2</sup>|
| <code>curve_fit</code>|area distribution fit parameters||
| <code>alteration</code>|quantitative alteration degree||



In [1]:
# Check if the summaries datafile needs to be regenerated on the basis of missing file or request.
if not os.path.exists(file_prepend + "-summary.csv") or FORCE_OVERWRITE:
    data = dict()
    # Iterate through samples and their area data.
    for sample,sample_area_data in areas_data.items():
        # Load patch areas.
        patch_areas = sample_area_data["patch_areas"]
        # Load area studied.
        area_studied = sample_area_data["area_studied"]
        # Load size of largest grain.
        largest_grain = sample_area_data["largest_grain"]
        # Compute distribution parameters for patch areas.
        # Note 99 rather than 100 as bin_values takes the number of bins rather than bin edges.
        counts,_,midpoints = bin_values(patch_areas,0.05,99)

        # Construct summary dataframe for each sample.
        data[sample] = {"convhull":area_studied, # study area
                        "n":len(patch_areas), # number of discrete reflectors after filtering
                        "total_area":sum(patch_areas), # area of reflectors after filtering
                        "largest":largest_grain, # largest continuous reflector patch area
                        "curve_fit":fit_exp_log_y(midpoints,counts)}

        # Degree of alteration assigned to each section.
        # Note: alteration_degree is imported from plotting.py
        try:
            data[sample]["alteration"] = alteration_degree[sample]
        except KeyError:
            pass

    # Convert dictionary to pandas dataframe.
    df = pd.DataFrame.from_dict(data,orient="index")
    # Save pandas dataframe to .csv file.
    df.to_csv(file_prepend + "-summary.csv")

### Comparison Plotting



After obtaining this data, comparisons can be plotted.

-   In some cases, derived parameters (that are normalised to the area studied) are more useful for comparing between sections.
    -   Reflector coverage area &rarr; reflector coverage percentage.
    -   Reflector count &rarr; reflector number density.
-   Only sections that are partially (0) or heavily (1) altered will be considered in the comparison.



In [1]:
# Force load from .csv file so that list processing is standardised.
df = pd.read_csv(file_prepend + "-summary.csv",index_col=0)
# Derived parameters that are more logical to compare between sections.
df["reflector_percentage"] = df["total_area"]/df["convhull"] * 100
df["number_density"] = df["n"]/df["convhull"]

# Look at only sections that have an alteration index of 1 (heavy) or 0 (partly).
df = df[(df["alteration"]==1) | (df["alteration"]==0)]

######################################################
# Comparison between aggregated reflector properties #
######################################################
fig,axs = plt.subplots(1,3,constrained_layout=True,figsize=(9,6))

# Plot point for each sample's property.
axs[0].scatter(df["alteration"],df["largest"])
axs[1].scatter(df["alteration"],df["number_density"])
axs[2].scatter(df["alteration"],df["reflector_percentage"])

# Label the sample referred to by each point.
for s,row in df.iterrows():
    x = row["alteration"]
    axs[0].text(x,row["largest"],s)
    axs[1].text(x,row["number_density"],s)
    axs[2].text(x,row["reflector_percentage"],s)

# Label the plots with which parameter is being compared.
axs[0].set_ylabel("Largest reflector area /mm$^2$")
axs[1].set_ylabel("Reflector number density /mm$^-2$")
axs[2].set_ylabel("Reflector coverage /%")

# Label the plots with the degree of alteration represented by plotted samples.
[ax.set_xlabel("Degree of alteration") for ax in axs]
[ax.set_xticks([0,1],["medium","high"]) for ax in axs]

plt.suptitle("Reflector parameter comparisons between\nmoderately and highly altered rocks")
save_figure(os.path.join("imgs",file_prepend + "-refl-param-comparison.png"))

#############################################
# Comparison between area distribution fits #
#############################################
fig,axs = plt.subplots(1,2,constrained_layout=True,figsize=(6,6))

# Load curve fit data.
curve_fits = np.array(json.loads("[" + ",".join(df["curve_fit"]) + "]"))

# Plot point for each sample's property.
axs[0].scatter(df["alteration"],curve_fits[:,0]/df["n"])
axs[1].scatter(df["alteration"],curve_fits[:,1])

# Label the plots with which parameter is being compared.
axs[0].set_ylabel("a/n")
axs[1].set_ylabel("b")

# Label the sample referred to by each point.
for i,alt in enumerate(zip(curve_fits[:,0]/df["n"],curve_fits[:,1])):
    s = df.iloc[i].name
    x = df.iloc[i]["alteration"]
    axs[0].text(x,alt[0],s)
    axs[1].text(x,alt[1],s)

# Label the plots with the degree of alteration represented by plotted samples.
[ax.set_xlabel("Degree of alteration") for ax in axs]
[ax.set_xticks([0,1],["medium","high"]) for ax in axs]

plt.suptitle("Fit parameter values in area distribution curve fit of format: $10^{a \cdot \exp(b x)}$")
save_figure(os.path.join("imgs",file_prepend + "-area_fit_param_comp.png"))
plt.show()

None

For the area distribution curve fits, and interpretation of the parameters' meanings are:

-   $a$: height of the distribution at the start such that $a/n$ is the height normalised by the number of reflectors (to permit comparison between sections). The larger $|a/n|$ is, the taller the start of the distribution relative to higher values.
-   $b$: measure of "decay" rate of the negative exponential distribution. The larger $|b|$ is, the narrower the distribution.



#### Reflector Parameter Comparison



<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-refl-param-comparison.png"></th>
<th><img src="./imgs/modified-10-refl-param-comparison.png"></th>
<th><img src="./imgs/modified-20-refl-param-comparison.png"></th>
</tr>
</table>



##### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:26&gt;</span></span>
Observations:

-   There's a narrowing of the range of values towards the smaller end for the largest parameter area with increasing alteration. This narrowing is most pronounced at 20x20 px dilation-erosion.
-   The reflector number density appears to also broaden in range with increasing alteration. This effect is clearest without dilation erosion, and is roughly equally less clear for 10x10 and 20x20 px dilation-erosion.
-   The reflector coverage density appears to broaden in range with increasing alteration but only clearly so at 20x20 px dilation-erosion.

Interpretation:

-   Due to the greater effect of heterogeneity on larger grains, the difference in largest grain sizes can't be confidently interpreted.
-   Broadening of number density and coverage suggests that increasing alteration can either have little effect on reflector number density, or can increase it.
-   The effect of different amounts of dilation-erosion is relatively important in determining how clear these changes in range are.



#### Area Distribution Comparison



<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-area_fit_param_comp.png"></th>
<th><img src="./imgs/modified-10-area_fit_param_comp.png"></th>
<th><img src="./imgs/modified-20-area_fit_param_comp.png"></th>
</tr>
</table>



##### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:37&gt;</span></span>
Observations:

-   $a/n$ generally broadens with increasing alteration. The amount of dilation-erosion has little effect on this.
-   $b$ generally decreases lower magnitudes with increasing alteration, with this effect being more pronounced with increasing dilation-erosion.

Interpretations:

-   Increasing alteration can change the relative size of the lowest area bin in different directions.
-   Increasing alteration generally broadens the area distribution, with this effect being more obvious with increasing dilation-erosion.



## Sample Property Aggregation



Area distributions can be aggregated and differenced to make inferences on the grain population produced with increasing hydration.

Looking at just the partially vs heavily altered sections (as the very heavily altered section just has one entry and is uncertain anyway):



In [1]:
# Overwriting the imported sample list with just the samples of interest (i.e. that have alteration indices of either 0 or 1).
alteration_degree = {k:v for k,v in alteration_degree.items() if v in [0,1]}

Loading area data and defining how it's being binned:



In [1]:
with open(file_prepend + "-areas.json") as infile:
    data = json.load(infile)

# Hardcoded maximum area to define bins with.
max_area = 0.05 # mm^2
bins = np.linspace(0,max_area,100)
# Compute bin midpoints.
midpoints = (bins[1:] + bins[:-1])/2
# Function to normalise data.
norm = lambda x : np.array(x)/sum(x)

Grouping normalised area distributions by degree of alteration, with each distribution weighted by how much area was studied to produce the distribution.



In [1]:
# Declare dictionary in which data will be aggregated.
grouped_data = dict()
# Iterate through sample data.
for key,area_data in data.items():
    # Extract areas data.
    areas = area_data["patch_areas"]
    # Extract the area studied.
    studied_area = area_data["area_studied"]
    # Check if the sample is of interest.
    if key in alteration_degree:
        # If so, extract the degree of alteration of the sample.
        alteration = alteration_degree[key]
        # Check if the degree of alteration of interest already has a preallocated data structure in the top-level dictionary dataframe.
        if not alteration in grouped_data:
            # If not, create this data structure.
            grouped_data[alteration] = {"distribution":[],
                                        "n":0}
        # Compute area distribution via histogram.
        counts,_ = np.histogram(areas,bins=bins)
        # Normalise the distribution.
        normed_counts = norm(counts)
        # Weight the distribution by the amount of area studied to produce that distribution.
        weighted_counts = studied_area * normed_counts
        # Store the distribution.
        grouped_data[alteration]["distribution"].append(weighted_counts)
        # Add to the number of reflector patches considered for sections of the active degree of alteration.
        grouped_data[alteration]["n"] += len(areas)

# Aggregate and normalise the distributions.
partially_altered = norm(np.sum(np.array(grouped_data[0]["distribution"]),axis=0))
heavily_altered = norm(np.sum(np.array(grouped_data[1]["distribution"]),axis=0))

Fitting a combined exponential and order 1 polynomial decay function to the distributions, and then saving the results of the fit to permit later investigation of the robustness of difference of distributions.



In [1]:
# Only fit to positive values (i.e. where the count is not zero).
fitting_p = partially_altered>0
fitting_h = heavily_altered>0

# Determine fit parameters.
popt_p,_ = curve_fit(exp_with_first_order_p_func,
                     midpoints[fitting_p],np.log10(partially_altered[fitting_p]))
popt_h,_ = curve_fit(exp_with_first_order_p_func,
                     midpoints[fitting_h],np.log10(heavily_altered[fitting_h]))

# Save fit parameters.
with open(file_prepend + "-distribution_fits.json","w") as outfile:
    json.dump({"partial":popt_p.tolist(),
               "heavy":popt_h.tolist(),
               "bins":bins.tolist()},
              outfile)

### Plotting Aggregated Distributions



In [1]:
# Plot the aggregated area distribution for partially altered samples, as well as the fit.
plt.stairs(partially_altered,bins,label="partially",color="b")
plt.plot(midpoints,10**exp_with_first_order_p_func(midpoints,*popt_p),c="b")
# Plot the aggregated area distribution for heavily altered samples, as well as the fit.
plt.stairs(heavily_altered,bins,label="heavily",color="g")
plt.plot(midpoints,10**exp_with_first_order_p_func(midpoints,*popt_h),c="g")
# Set y scale to log.
plt.gca().set_yscale("log")
# Label axes.
plt.xlabel("Area /mm$^2$")
plt.ylabel("Frequency")
# Display legend.
plt.legend()

save_figure(os.path.join("imgs",file_prepend+"-partially-vs-heavily-altered.png"))
plt.show()

None

Generally speaking, these fits are not great &#x2026;

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-partially-vs-heavily-altered.png"></th>
<th><img src="./imgs/modified-10-partially-vs-heavily-altered.png"></th>
<th><img src="./imgs/modified-20-partially-vs-heavily-altered.png"></th>
</tr>
</table>



#### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:48&gt;</span></span>
Observations:

-   These fits aren't great (even ignoring the semilog nature of these plots)

Interpretation:

-   A better fit function may be needed - or manually drawing continuous distributions?



### Plotting Differenced Distributions



Plotting the difference in heavily altered distribution and partially altered distribution to characterise the change following increasing alteration.



In [1]:
# Compute difference in distributions.
diff = heavily_altered-partially_altered
# Plot horizontal line at y=0.
plt.axhline(0,c="lightblue",linestyle="--")
# Plot difference in distributions
plt.stairs(diff,bins,label="heavily-partially altered freqs.",color="k")
# Label axes.
plt.xlabel("Area /mm$^2$")
plt.ylabel("Heavily minus Partially altered Freq. Diff.")

save_figure(os.path.join("imgs",file_prepend+"-heavily-minus-partially-altered.png"))
plt.show()

None

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-heavily-minus-partially-altered.png"></th>
<th><img src="./imgs/modified-10-heavily-minus-partially-altered.png"></th>
<th><img src="./imgs/modified-20-heavily-minus-partially-altered.png"></th>
</tr>
</table>



#### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:49&gt;</span></span>
Observations:

-   There's a relatively consistent observation across the range of dilation-erosions tested that there's a decrease in the proportion of some finer grain size, with an increase in grains just coarser, and that increase decaying with increasing grain size up to ~0.01 mm<sup>2</sup>.
    -   Where no dilation-erosion is applied, there's an increase in the finest grain size fraction considered, but when 10x10 or 20x20 px dilation-erosion is applied, the finest fraction experiences a significant decrease.

Interpretations:

-   The finest grains are dissolved and reprecipitated on other smaller grains to increase their grain size.
    -   This is different to the finest grains being preferentially dissolved as that would increase the proportion of all coarser grains.
    -   I.e. the decaying increase in grain proportions between ~0.002 and 0.01 mm<sup>6</sup> is likely related to precipitation effects.



### Testing the Robustness of the Difference in Distributions



The difference is distributions can be tested by a bootstrapping method assuming the fitted distributions accurately reflect the parent distribution of the observed distributions:

1.  Generate a subsample of each fitted distribution, where the size of each subsample corresponds to the size of the dataset used for each aggregated distribution.
2.  Use some statistical method of determining whether two empirical (i.e. discrete) distributions are different to calculate the confidence with this the distributions can be described as different. Some methods identified are described in table [1](#org6960067). Based on my interpretation of these tests, the Kolmogorov-Smirnov test should be the most useful in quantifying whether the distributions are likely different or not. However it's scipy implementation is only valid for continuous distributions *sensu stricto*.
3.  Repeat this process of subsampling and comparing many times until a relatively smooth distribution of confidences is produced.


| Method|H0 (for two samples)|
|---|---|
| Student T Test|Averages don't differ|
| (2 Sample) Kolmogorov-Smirnov Test|Distributions don't differ (both in location and shape)|
| Mann-Whitney U Test|There's an equal probability of a randomly selected value from one distribution being greater than vs less than a randomly selected value from the other distribution|



In [1]:
def bootstrap_test_difference(difference_test,test_repeats,size1,p1,size2,p2):
    # Allocate lists into which test outcomes are to be saved.
    difference_test_results_11 = []
    difference_test_results_12 = []

    # Repeat the analysis the desired number of times.
    for i in range(test_repeats):
        # Random sample based on the first distribution of the first sample's size.
        rand_a = rng.choice(midpoints,size=size1,p=norm(p1))
        # Random sample based on the first distribution of the second sample's size.
        rand_b = rng.choice(midpoints,size=size2,p=norm(p1))
        # Random sample based on the second distribution of the second sample's size.
        rand_c = rng.choice(midpoints,size=size2,p=norm(p2))
        # Performing the difference test for random samples extracted from the same distribution.
        difference_test_results_11.append(list(difference_test(rand_a,rand_b)))
        # Performing the difference test for random samples extracted from different distributions.
        difference_test_results_12.append(list(difference_test(rand_a,rand_c)))

    # Convert test outcome lists into numpy arrays.
    difference_test_results_11 = np.array(difference_test_results_11)
    difference_test_results_12 = np.array(difference_test_results_12)
    return difference_test_results_11,difference_test_results_12

Scipy tests generate a test statistic, as well as a P-value. The null hypotheses for the various tests in the table effectively boil down to "the distributions are the same". In order to reject this null hypothesis, the observed P-value must be below a critical value - this is taken (somewhat arbitrarily) at 0.05. Therefore, in the distribution of P-values (generated by bootstrapping), the percentage of P-values below 0.05 is a proxy for the likelihood that the two distributions (derived from data of almost-certainly unequal sizes) are different.

-   To avoid this taking an excessively long time, an undersampling factor is introduced, which reduces the size of both samples by a common factor (through division by the undersampling factor).
-   The larger this undersampling factor is, the greater the effect of randomness on the distributions. However, the fundamental nature of the test should remain.
-   Regardless, keeping the undersampling factor as close to 1 as feasible is preferred.



In [1]:
# For distribution testing functions.
import scipy.stats as sps

#####

test_repeats = 5000
p_crit = 0.05

#####

# Factor by which to reduce the number of items in each sample.
undersample_factor = 20
# Number of items in each sample based on how many were present in the data.
n_partly_altered = int(grouped_data[0]["n"]/undersample_factor)
n_heavily_altered = int(grouped_data[1]["n"]/undersample_factor)

print(f"Number in first sample: {n_partly_altered}\nNumber in second sample: {n_heavily_altered}")

# Declare the types of statistical difference tests to apply.
difference_tests = [sps.ttest_ind,sps.ks_2samp,sps.mannwhitneyu]

# Load the distribution fits data.
with open(file_prepend + "-distribution_fits.json") as infile:
    data = json.load(infile)
# Extract fit parameters.
fit_p = data["partial"]
fit_h = data["heavy"]
# Extract bins used to produce the distributions that the fit parameters were derived from.
bins = np.array(data["bins"])
# Compute the midpoints of these bins.
midpoints = (bins[:-1] + bins[1:])/2

# Function to construct a discrete probability distribution (for specified x values) applicable to the fit function used to produce the fit parameters above.
p_x = lambda x,fit : 10**exp_with_first_order_p_func(x,*fit)
# Various partial functions:
p_x_p = lambda x : p_x(x,fit1)
p_x_h = lambda x : p_x(x,fit2)
p = lambda fit : p_x(midpoints,fit)
# Discrete probability distributions.
p_p = p(fit_p)
p_h = p(fit_h)

# Function to normalise data.
norm = lambda x : x/sum(x)

# Initiate random number generator.
rng = np.random.default_rng()

# Specify top-level plot layout.
fig,axs_0 = plt.subplots(2,len(difference_tests),constrained_layout=True,figsize=(20,20))

# Iterate through the statistical difference tests.
for i,difference_test in enumerate(difference_tests):
    # Obtain bootstrapped results for the active difference test.
    difference_test_results_11,difference_test_results_12 = bootstrap_test_difference(difference_test,test_repeats,n_partly_altered,p_p,n_heavily_altered,p_h)

    # Define number bins for the distribution of p-values.
    p_val_bins = np.linspace(0,1,101)

    # Extract p-values for the results of different combination of distributions (same distribution vs different distributions).
    p_val11 = difference_test_results_11[:,1]
    p_val12 = difference_test_results_12[:,1]

    # Isolate a column in the plot.
    axs = axs_0[:,i]
    # Iterate through the p-values for each combination of distributions.
    for p_vals,ax in zip([p_val11,p_val12],axs):
        # Plot the distribution for each collection of p-values.
        ax.hist(p_vals,p_val_bins)
        # Plot a vertical line at the critical p-value.
        ax.axvline(p_crit,color="r")
        # Label axes.
        ax.set_xlabel("p value")
        ax.set_ylabel("Count")
        # Compute and display the percentage of observed p-values below critical.
        ax.set_title("%s; p<%s = %.2f%%" % (difference_test.__name__,p_crit,sum(p_vals<p_crit)/len(p_vals)*100))

save_figure(os.path.join("imgs",file_prepend + "-diff-test.png"))
plt.show()

None

In the following figures, the top row contains the test results from samples drawn from the same distribution (the partially altered distribution), and the bottom row from different distributions (partially altered vs heavily altered).

<table>
<tr>
<th style="text-align:center">No dilation-erosion</th>
<th style="text-align:center">10x10 px kernel dilation-erosion</th>
<th style="text-align:center">20x20 px kernel dilation-erosion</th>
</tr>
<tr>
<th><img src="./imgs/unmodified-diff-test.png"></th>
<th><img src="./imgs/modified-10-diff-test.png"></th>
<th><img src="./imgs/modified-20-diff-test.png"></th>
</tr>
</table>



#### Discussion



<span class="timestamp-wrapper"><span class="timestamp">&lt;mer. déc.  6 2023 15:54&gt;</span></span>
Observations:

-   Out of all the tests, only the KS-test is able to clearly identify the same distributions as being the same - i.e. peak in p-value distributions at high p (the other tests appear to produce an even distribution of p-values, which suggests the same distributions are equally as likely to be the same as to being different).
    -   Therefore, the results of the KS-test are probably the most reliable in this circumstance.
-   All tests seem to suggest that the partially vs heavily altered distributions are different for all amounts of dilation-erosion.
    -   However, the t-test seems to be quite sensitive to differences in fit shapes produced by different amounts of dilation-erosion.

Interpretations:

-   Subsamples of the fitted continuous distributions of the partially vs heavily altered empirical area distributions have a high likelihood of being different. Therefore - assuming the empirical area distributions represent subsamples of their respective fitted continuous distributions - the empirical area distributions are likely different as well.

